In [1]:
from binance import *
from ta import *
from backtest import *

import matplotlib.pyplot as plt

In [2]:
symbol = 'BTC/USDT'
freq = '1h'

In [3]:
plot_prices([symbol[:-5]], freq)

In [4]:
dict_df = get_dict_df(symbols=[symbol], freq=freq, columns=['close', 'open'])

dict_df = add_dict_df_emas(dict_df, lengths=[12, 21])

df = dict_df[symbol]

df

,close,open,EMA_12,EMA_21,Signal_EMA_12_21,Delta_EMA_12_21,Rel_Delta_EMA_12_21,Ratio_EMA_12_21
timestamp,,,,,,,,
2023-01-01 00:00:00,16529.67,16541.77,NaN,NaN,0,NaN,NaN,NaN
2023-01-01 01:00:00,16551.47,16529.59,NaN,NaN,0,NaN,NaN,NaN
2023-01-01 02:00:00,16548.19,16551.47,NaN,NaN,0,NaN,NaN,NaN
2023-01-01 03:00:00,16533.04,16548.19,NaN,NaN,0,NaN,NaN,NaN
2023-01-01 04:00:00,16521.85,16533.04,NaN,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,42619.04,42656.69,42533.525945,42461.150958,0,72.374986,0.001704,0.170450
2023-12-31 20:00:00,42558.01,42619.04,42537.292722,42469.956326,0,67.336397,0.001586,0.158551
2023-12-31 21:00:00,42520.72,42558.02,42534.743073,42474.571205,0,60.171868,0.001417,0.141666


In [5]:
# df_plot = df[['close', 'Signal_EMA_12_21']]
df_plot = df[['close','EMA_12', 'EMA_21']]

plot_df(df_plot)

In [6]:
trades_prev, trades_next, trades_2prev, trades_3prev = get_trades(df, 'Signal_EMA_12_21', freq)

In [7]:
# buy and hold
entry = df['close'].iloc[0]
exit = df['close'].iloc[-1]

f'Return: {round((exit/entry-1)*100,2)} %'

'Return: 155.8 %'

In [8]:
value_list = []
compound_value_list = []
range_list = [x/100 for x in range(1, 50)]
for pct in range_list:
    df_rel_delta = df[['open', 'Ratio_EMA_12_21']].copy()
    df_rel_delta['Signal_Ratio_EMA_12_21'] = np.where((df_rel_delta['Ratio_EMA_12_21'] < pct) & (df_rel_delta['Ratio_EMA_12_21'].shift(1) > pct), -1,
                                            np.where((df_rel_delta['Ratio_EMA_12_21'] > -pct) & (df_rel_delta['Ratio_EMA_12_21'].shift(1) < -pct), 1, 0))
    df_rel_delta.drop(columns='Ratio_EMA_12_21', inplace=True)
    _, trades_next_rel_delta, _, _ = get_trades(df_rel_delta, 'Signal_Ratio_EMA_12_21', freq)

    value, df_pnl = pnl(trades_next_rel_delta, verbose=False)
    value_list.append(value)

    compound_value, df_compound_pnl, df_compound_return = pnl_compund(trades_next_rel_delta, verbose=False)
    compound_value_list.append(compound_value)

df_roi = pd.DataFrame({'pct': range_list, 'value': value_list})
df_roi.set_index('pct', inplace=True)
plot_df(df_roi)
# plot_df(df_pnl)

df_compound_roi = pd.DataFrame({'pct': range_list, 'compound_value': compound_value_list})
df_compound_roi.set_index('pct', inplace=True)
plot_df(df_compound_roi)
# plot_df(df_compound_pnl)
# plot_df(df_compound_return*100)